In [3]:
!pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 8.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
import re
import pickle
import string
import unicodedata
from random import randint

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from wordcloud import STOPWORDS, WordCloud

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding, TimeDistributed

In [8]:
!pip install contractions==0.1.73

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from contractions import contractions_dict

for key, value in list(contractions_dict.items())[:10]:
    print(f'{key} == {value}')

I'm == I am
I'm'a == I am about to
I'm'o == I am going to
I've == I have
I'll == I will
I'll've == I will have
I'd == I would
I'd've == I would have
Whatcha == What are you
amn't == am not


In [10]:
# Using TPU

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1731171049.355551     165 service.cc:145] XLA service 0x594f8ee51650 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731171049.355606     165 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1731171049.355611     165 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1731171049.355614     165 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1731171049.355617     165 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1731171049.355620     165 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1731171049.355623     165 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1731171049.355632     165 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1731171049.355635     165 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


Loading data

In [11]:
train_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")

In [12]:
train_data

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [13]:
test_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")

In [14]:
test_data

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [15]:
val_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv")

In [16]:
val_data

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...
...,...,...,...
13363,e93f721ba4949f21f33549c4a21d55ff456af979,All shops will be allowed to offer ‘click and ...,Shops won't have to apply for planning permiss...
13364,8df19a570ad14119a7d00f3bbe864fedf8c1691d,Mo Farah has had his nationality called into q...,Mo Farah broke the European half-marathon reco...
13365,2fdd5f89aa26e91ceea9b0ef264abfcfc3e6fa2e,Wolves kept their promotion hopes alive with a...,Wolves are three points off the play-off place...
13366,530d7b18d7a715b368b0745f9dfebfe353adeda8,A Brown University graduate student has died ...,"Hyoun Ju Sohn, a 25-year-old doctoral student,..."


In [17]:
def expand_contractions(text, contraction_map=contractions_dict):
    # Using regex for getting all contracted words
    contractions_keys = '|'.join(contraction_map.keys())
    contractions_pattern = re.compile(f'({contractions_keys})', flags=re.DOTALL)

    def expand_match(contraction):
        # Getting entire matched sub-string
        match = contraction.group(0)
        expanded_contraction = contraction_map.get(match)
        if not expand_contractions:
            print(match)
            return match
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


expand_contractions("y'all can't expand contractions i'd think")

'you all cannot expand contractions id think'

In [21]:
# Converting to lowercase
train_data.highlights = train_data.highlights.apply(str.lower)
train_data.article = train_data.article.apply(str.lower)

train_data.sample(5)

,id,article,highlights
282044,f958a776d4a5117ce05670efa29a78db3e6a293a,(cnn) -- australia's marc leishman and sergio ...,marc leishman and sergio garcia lead the field...
276920,f2c1d5149f1d71d059022487bd56249aee9c3b1c,(cnn) -- as part of cnn's defining america pro...,can you judge a book by its cover? find out.\n...
49159,8ad7f933f911820f748e656da270ac9d3e370393,"by . naomi greenaway . for most women, shiftin...","maggie groome, 35, weighed 20st before her pre..."
123570,2bbd0ebf5ecc793e5fc911a804db21e83d44aa53,london (cnn) -- sub-zero temperatures continue...,the heart of the cold air is centered over eas...
112274,1cd4b5eab5e146d9f1a1ccbc6a2a1c67e6032135,(cnn) -- the remaining scaffolding around the ...,a 2011 earthquake damaged the washington monum...


In [22]:
# Converting to lowercase
test_data.highlights = test_data.highlights.apply(str.lower)
test_data.article = test_data.article.apply(str.lower)

test_data.sample(5)

,id,article,highlights
3028,bb753d92e432499343e35904c9926b3c43941a4b,real sociedad remain hopeful over a deal for b...,danny ings' contract expires in the summer and...
9175,8f5eb904e400e14968ed02e691819161fe9a82cb,(cnn)the fbi is investigating a possible isis-...,officials say the investigation originated fro...
4648,8f1d0d3052677902ce6282885dcc81a286b07fa3,rangers boss stuart mccall insists it will not...,rangers are currently second in the championsh...
546,7d8389d498d79d1bcc367fa3c83e592386f2ce06,around one in seven couples suffer with infert...,around one in seven couples suffer with infert...
10868,203886369feea77bbc35715e6d7e518b751f57de,"(cnn)seventy years ago, anne frank died of typ...",museum: anne frank died earlier than previousl...


In [23]:
# Converting to lowercase
val_data.highlights = val_data.highlights.apply(str.lower)
val_data.article = val_data.article.apply(str.lower)

val_data.sample(5)

,id,article,highlights
112,0875a3a89684ad75bbb6dd9d5514ddd678828dca,should rangers win promotion to the premiershi...,"rangers will owe newcastle united £500,000 if ..."
129,144e128132cd522d82aa0b1e21ed83848e7e4757,modou barrow has joined nottingham forest on l...,swansea manager garry monk wants modou barrow ...
10091,288af62a9387117928bfb42b474dd8bc0066f4d4,the personal details of 31 world leaders inclu...,immigration official sent data to asian cup fo...
10246,0428409fae9bcabd1e4dab41e6e60e03c61e25c0,a man who led a kansas commune that collected ...,"daniel perez, 55, led a kansas commune that co..."
3295,7187fe2968cf29d3fdb19228bfb451099743ed81,england and wales cricket board managing direc...,england fell to a 15-run defeat by bangladesh ...
